In [1]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
Please enter the verification code: Access token retrieved correctly.


In [2]:
!pip install tqdm

    100% |████████████████████████████████| 51kB 2.3MB/s 


In [3]:
import csv
import nltk
from tqdm import tqdm
import numpy as np
from keras.preprocessing import sequence

Using TensorFlow backend.


In [0]:
# How many lines to process
line_limit = 399998

In [5]:
!mkdir -p drive
!google-drive-ocamlfuse drive

fuse: mountpoint is not empty
fuse: if you are sure this is safe, use the 'nonempty' mount option


In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import nltk
from nltk.corpus import stopwords
import string
nltk.download('stopwords')
nltk.download('punkt')

%matplotlib inline
%config InlineBackend.figure_format='retina'


[nltk_data] Downloading package stopwords to /content/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /content/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [7]:
with open('drive/NUS_Project/train_new.csv', 'r') as f:
    reader = csv.reader(f)
    
    lines = []
    
    for row in tqdm(reader):
        lines += [row]
        
    lines = lines[1:line_limit+1]

400001it [00:03, 117797.17it/s]


In [8]:
X = []
Y = []
unicode_errors = 0

for label, sentence in tqdm(lines):
    try:
        Y += [label]
        X += [nltk.word_tokenize(sentence.decode('ascii', 'ignore'))]
    except UnicodeDecodeError:
        unicode_errors += 1
        print 'Unicode errors:', unicode_errors

100%|██████████| 399998/399998 [10:50<00:00, 615.11it/s]


In [9]:
counts = dict()

for i in tqdm(range(len(X))):
  for word in X[i]:
    word = word.lower()
    if word not in counts:
      counts[word] = 0
    counts[word] += 1

100%|██████████| 399998/399998 [00:24<00:00, 16300.17it/s]


In [0]:
freq_sorted = [k for v, k in sorted([(v, k) for k, v in counts.iteritems()], reverse=True)]

In [15]:
vocabulary = ['out_of_vocab'] + freq_sorted[:9999]
print len(vocabulary)

10000


In [0]:
idx2w = [(i, v) for i, v in zip(range(len(vocabulary)), vocabulary)]
w2idx = [(v, i) for i, v in idx2w]

w2idx = dict(w2idx)
idx2w = dict(idx2w)

In [0]:
X_vec = []
Y_vec = []

for i in tqdm(range(len(X))):
    X_vec += [[]]
    Y_vec += [np.zeros(3)]
    
    for w in X[i]:
        if w.lower() not in vocabulary:
          X_vec[-1] += [w2idx['out_of_vocab']]
        else:
          X_vec[-1] += [w2idx[w.lower()]]
    
    Y_vec[-1][int(Y[i])] = 1

In [0]:
X_vec = sequence.pad_sequences(X_vec, maxlen=125)
Y_vec = np.array(Y_vec)

This brings back memories

You want to write a crossover script featuring Phoenix Wright and the Ghost Trick BGM?

In [15]:
print 'Vocabulary size:', len(vocabulary)

Vocabulary size: 10000


In [0]:
np.save("X.npy", X_vec)
np.save("Y.npy", Y_vec)

========================================================================

In [0]:
!pip install -q keras

In [0]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Embedding, LSTM, Bidirectional
import numpy as np

In [0]:
vocab_size = 10000
maxlen = 125

In [20]:
model = Sequential()
model.add(Embedding(vocab_size, 128, input_length=maxlen))
model.add(Bidirectional(LSTM(128, return_sequences=True)))
model.add(Bidirectional(LSTM(128)))
model.add(Dropout(0.5))
model.add(Dense(300, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(300, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(3))

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 125, 128)          1280000   
_________________________________________________________________
bidirectional_1 (Bidirection (None, 125, 256)          263168    
_________________________________________________________________
bidirectional_2 (Bidirection (None, 256)               394240    
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 300)               77100     
_________________________________________________________________
dropout_2 (Dropout)          (None, 300)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 300)               90300     
__________

In [0]:
model.compile('adam', 'categorical_crossentropy', metrics=['accuracy'])

In [22]:
from keras import backend as K
K.tensorflow_backend._get_available_gpus()

['/job:localhost/replica:0/task:0/device:GPU:0']

In [23]:
x_train = np.load("X.npy")
y_train = np.load("Y.npy")

model.fit(x_train, y_train, batch_size=64, epochs=4)

Epoch 1/4
5000/5000 [==============================] - 85s 17ms/step - loss: 1.2395 - acc: 0.5994
Epoch 2/4
5000/5000 [==============================] - 83s 17ms/step - loss: 0.9323 - acc: 0.6138
Epoch 3/4
5000/5000 [==============================] - 83s 17ms/step - loss: 0.8918 - acc: 0.6204
Epoch 4/4
5000/5000 [==============================] - 83s 17ms/step - loss: 0.9164 - acc: 0.6200


In [0]:
vect_t2 = hstack([vect_t, exclaim_t])
#vect = hstack([vect2, question])
vect_t2 = csr_matrix(vect_t2)
vect_t2

In [0]:
y_t_pred = lin_clf.predict_class(vect_t2)


In [0]:
df_ret = pd.DataFrame(y_t_pred)
df_ret.index += 1 
df_ret['ID'] = df_ret.index
df_ret.columns = ['label', 'ID']
df_ret = df_ret[['ID', 'label']]
df_ret

In [0]:
df_ret.to_csv("result_svm_ch.csv", index=False)

In [0]:
from google.colab import files
files.download('result_svm_ch.csv')